# Modelo de lenguaje

In [8]:
import numpy as np
import random
import pickle
import pprint as p
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

### Definición de la red

In [2]:
class ModeloB:
    
    def __init__(self, dim_in  = 2, h_layers = [3 , 5], dim_out = 2, lr = 0.02):
        """Inicialización de las matrices de la red neuronal y otros parámetros """
        #Establecida para experimentación
        np.random.seed(505) 
        random.seed(505)
        
        self.dim_in  = dim_in # Dimensión de la entrada
        self.dim_h_layers = len(h_layers) # Número de capas ocultas
        self.dim_out = dim_out # Dimensión capa de salida

        self.lr = lr #Learning Reate

        self.layers = h_layers #Neuronas por capas internas
        self.layers.insert(0,dim_in)
        self.layers.append(dim_out)
        
        #Definición de funciones de activación
        self.ident = lambda x : x
        self.tanh = lambda x : np.tanh(x)
        self.soft_max = lambda x : np.exp(x - np.max(x))/np.exp(x - np.max(x)).sum(0, keepdims=True) #Cero porque son vectores columna
        
        #Listas de matrices que almacenarán los pesos
        self.weigths = list() 
        self.bias = list()
        self.Hs = list()
        
        #Inicialización de matrices
        for k in range(self.dim_h_layers + 1):
            self.weigths.append(np.random.rand(self.layers[k+1],self.layers[k])/np.sqrt(self.layers[k+1]))#Pa que converja rápido
            self.bias.append(np.ones((self.layers[k+1],1)))
            self.Hs.append(np.zeros((self.layers[k+1],1)))

    def _fordward(self, x):
        """Proceso hacia adelante para el cálculo de la predicción"""
        #Obtiene el embedding
        self.Hs[0] = self.weigths[0][:,x].reshape((100,1))
        #Calcula el valor de la capa oculta
        self.Hs[1] = self.tanh(np.dot(self.weigths[1],self.Hs[0]) + self.bias[1])
        #Calcula el valor de la capa de salida (predicción)
        self.Hs[2] = self.soft_max(np.dot(self.weigths[2],self.Hs[1]) + self.bias[2])     
        
    def _backward(self,y_idx,x_idx):
        """Proceso de en reversa para la propagación del error"""
        #Creación del vector de salida
        y = np.zeros((self.sizeVocaculary,1))
        y[y_idx] = 1
        
        #Errores por capa
        d_salida = self.Hs[-1] - y
        d_hidden =(1 - self.Hs[-2]**2) * np.dot(self.weigths[-1].T,d_salida)
        d_embedd =  np.dot(self.weigths[-2].T,d_hidden)

        #Actualización de pesos
        self.weigths[-1] -=  self.lr * np.outer(d_salida,self.Hs[-2])
        self.weigths[-2] -=  self.lr * np.outer(d_hidden,self.Hs[-3])
        self.weigths[-3][:,x_idx] -= self.lr * d_embedd.reshape(-1) 

        #Actualización de bias
        self.bias[-1] = self.bias[-1] - self.lr * d_salida
        self.bias[-2] = self.bias[-2] - self.lr * d_hidden
        
    def train(self, epochs = 5):
        """Proceso de entrenamiento utilizando Cross-Validation, se fija un número de épocas"""
        for epoch in range(epochs):
            for split in range(0,self.split):
                train_split, validation_split =  self.getTrainValidSplits(split)#Obtiene los conjuntos
                count = 0
                for k in train_split:
                    x = self.bigrams[k][0] #Obtención del índice de entrada asociado a la muestra k
                    y = self.bigrams[k][1] #Obtención del índice de salida asociado a la muestra k
                    self._fordward(x)
                    self._backward(y,x)
                    count += 1 
                    if (count%5000 == 0):
                        print('Epoch:', epoch ,' fold: ',split+1,' iteración: ',count,' / ',len(train_split))    
                self.validation(validation_split)
            self.crossValidation()
                
            
    def validation(self,validation_split):
        """Procedimiento para la validación del modelo"""
        sizeValid = len(validation_split)
        prediciones = np.zeros(sizeValid) 
        #Obtine la probabilidad de ver y después de x para cada muestra
        for k,i in zip(validation_split,range(sizeValid)):
            x = self.bigrams[k][0]
            y = self.bigrams[k][1]
            self._fordward(x)
            prediciones[i] = self.Hs[2][y]
        #self.perplejidad.append(np.prod(1/prediciones)**(1/sizeValid))
        self.entropia.append( -(1/sizeValid)*np.sum(np.log2(prediciones)))
        print("Entropía parcial (por k-fold): ", self.entropia[-1],"\n")
        
    def crossValidation(self):
        """Cálculo de la validación cruzada de una época"""
        self.entropiaEpoch.append(sum(self.entropia)*(1/self.split))
        self.entropia = list()
        print("Entropía cruzada (por época): ", self.entropia[-1],"\n")
        
    def predic(self, x):
        """Retorna el vector de probabilidades para la palabra representada por el índice x"""
        self._fordward(x)
        return self.Hs[2]
        
    def loadData(self, bigrams, sizeVocaculary):
        """Carga el conjunto de entrenamiento"""
        self.bigrams = bigrams
        self.sizeData = len(bigrams)
        self.sizeVocaculary = sizeVocaculary
        self.buildTrainValidationSplits(bigrams, 5)
        
    def buildTrainValidationSplits(self, bigrams, numsplit = 5):
        """Realiza una mezcla de los datos de entrenamiento, inicializa variables para validación"""
        self.indexes = [i for i in range (len (bigrams))]
        random.shuffle(self.indexes)#Mezcla los índices de cada muestra  
        self.split = numsplit
        self.sizeSplit = len(bigrams)//numsplit
        
        #self.perplejidad = list()
        self.entropia = list()
        self.entropiaEpoch = list()
        
    def getTrainValidSplits(self, k_fold):
        """Retorna el los conjuntos de entrenamiento y validación
           Indicados por el parámetro k_fold"""
        ini = (k_fold)*self.sizeSplit
        fin = (k_fold + 1)*self.sizeSplit
        
        if k_fold > 0 and k_fold<self.split-1:
            train_split = self.indexes[0:ini] + self.indexes[fin:]
            validation_split = self.indexes[ini:fin]
        elif k_fold == 0 :
            train_split = self.indexes[fin:]
            validation_split = self.indexes[0:fin]
        else:
            train_split = self.indexes[0:ini]
            validation_split = self.indexes[ini:]
        return train_split, validation_split
            
    def loadModel(self,file='model.pk'):
        """Carga de los pesos almacenados"""
        with open(file,'rb') as model_file:
            self.weigths = pickle.load(model_file)
            self.bias = pickle.load(model_file)
            #self.entropiaEpoch = pickle.load(model_file)
        print('¡Modelo cargado correctamente!')
    
    def export(self,file='model_2.pk'):
        """Almacena los pesos de la red"""
        with open(file,'wb') as model_file:
            pickle.dump(self.weigths, model_file)
            pickle.dump(self.bias, model_file)
            pickle.dump(self.entropiaEpoch, model_file)

### Carga de los bigramas

In [3]:
with open('Corpus.pk','rb') as corpus_file:
    dic_words = pickle.load(corpus_file)
    dic_index = pickle.load(corpus_file)
    vocabulario = pickle.load(corpus_file)
    bigrams = pickle.load(corpus_file)
    

### Ejecución de la red
La red se entrenó en varia épocas cargando y almacenando puntos de guardado, el learning rate inicial fue de 0.1 para 1 época, posteriormente se realizaron 10 épocas con un lr = 0.05, 8 épocas con lr= 0.002 y finalmente 3 con lr = 0.005 

In [4]:
dim = len(vocabulario)
model = ModeloB(dim, [100 , 300], dim, 0.005) #Más alto de 0.1 hace que no converga
model.loadData(bigrams, dim)
model.loadModel('model_18.pk')
#model.train(3)
#model.export('model_21.pk')

¡Modelo cargado correctamente!


### Las 10 probabilidades más altas por cada palabras 

In [10]:
@interact
def navegacion(word = sorted(list(dic_words.keys()))):
    palabra = dic_words[word]
    prediccion = model.predic(palabra)
    lis = [(dic_index[idx],prob) for idx,prob in zip (range(len(prediccion.T[0])),prediccion.T[0])]
    p.pprint(sorted(lis, key = lambda x: x[1], reverse=True)[:10])

interactive(children=(Dropdown(description='word', options=('1', '10', '100', '11', '12', '13', '16', '2', '20…

# Evaluación  del modelo

In [ ]:
model.loadData(bigrams, dim)
model.loadModel('model_18.pk')